In [19]:
# Importing the necessary libraries
from pymongo import MongoClient, GEOSPHERE
from pprint import pprint
from datetime import datetime

In [20]:
# Settingup Connection to MongoDB
username = "sk10689"
password = "sk10689"
host = "mongo-csgy-6513-spring.db"
auth_database = "sk10689"

In [21]:
client = MongoClient(f"mongodb://{username}:{password}@{host}/{auth_database}")

In [22]:
db = client[auth_database]
db

Database(MongoClient(host=['mongo-csgy-6513-spring.db:27017'], document_class=dict, tz_aware=False, connect=True), 'sk10689')

# Q2. Restaurant foreclosures in North Carolina – 75 Points

In [23]:
# Creating a geospatial index on the 'geometry' field of the 'durham' collection
db.durham.create_index([("geometry", GEOSPHERE)], background=True)

'geometry_2dsphere'

In [24]:
# Filtering restaurants in Durham with seating capacity of at least 100 and create a polygonal boundary around them
filter_restaurants = db.restaurants_durham.find({"Rpt_Area_Desc": "Food Service", "Seats": {"$gte": 100}})

polygon_coords = []

for row in filter_restaurants:
    if row['geolocation']:
        latitude, longitude = map(float, row['geolocation'].split(', '))
    else:
        continue
    
    polygon_coords.append([longitude, latitude])

In [25]:
polygon_coords.append(polygon_coords[0])

In [26]:
# Querying the 'durham' collection for all documents that fall within the polygon and return the count of 
#those documents
foreclosures_polygon = db.durham.find({
    "geometry":{
        "$geoWithin":{
            "$geometry":{
                "type":"Polygon",
                "coordinates":[polygon_coords]
            }
        }
    }
})

In [27]:
foreclosures_polygon.collection.count_documents({})

1948